# 10 Famous Cities of The American Continent Predominant Food Styles, A Data Analysis on The Restaurant Scenery and The Demographic Influence.

By Francisco Tosetto da Silva

For the entrepreneur / business person that is looking for a insight of a city food scenery, influences and preferences, or for the traveler that is looking for the famous, typical and / or traditional cuisine style on his destination, this analysis looks into the cuisine type scenery in venues of 10 famous cities in the American Continent.

This analysis focus on showing the top 3 most frequent styles of cuisine in restaurants from 10 famous cities located on the American Continent. To find out which types of cuisine are predominant in each place we look into around 100 food venues from these cities and see which types are more common. We also make an analysis on historical immigration data to find out the influences in the food scenery of these places.

This analysis utilyzes data from the free Foursquare Developer API.

#### About the data collection:
 This project will use the foursquare API to collect the cities venues information, as it can be seen below, as well as collecting data and information on the demography of the cities, searching for government information abaout immigration and historic constitution of the population to seek ties to the actual food scenery observed right now.

### Importing necessary libraries.

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import requests
import pandas as pd
import numpy as np
import folium
import random
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

### First we set up the cities that we want to collect data from and their location.

In [3]:
addresses = ['Vancouver, Canada', 'Montreal, Canada', 'Quebec City, Canada', 'Edmonton, Canada', 'Calgary, Canada',
             'Ottawa, Canada', 'Toronto, Canada', 'Winnipeg, Canada', 'Saskatoon, Canada', 'Whitehorse, Canada',
             'Yellowknife, Canada', 'Iqaluit, Canada', 'St. John\'s, Canada', 'Fredericton, Canada']
#'New York City, USA', 'Los Angeles, USA', 'São Paulo, Brazil',
 #            'Rio de Janeiro, Brazil', 'Mexico City, Mexico', 'Lima, Peru', 'Santiago, Chile', 'Buenos Aires, Argentina']
lat = []
lng = []
geolocator = Nominatim(user_agent = "my-application")
for i in addresses:
    location = geolocator.geocode(i)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(i, latitude, longitude))
    lat.append(latitude)
    lng.append(longitude)

The geograpical coordinate of Vancouver, Canada are 49.2608724, -123.1139529.
The geograpical coordinate of Montreal, Canada are 45.4972159, -73.6103642.
The geograpical coordinate of Quebec City, Canada are 46.8259601, -71.2352226.
The geograpical coordinate of Edmonton, Canada are 53.535411, -113.507996.
The geograpical coordinate of Calgary, Canada are 51.02532675, -114.049868485806.
The geograpical coordinate of Ottawa, Canada are 45.421106, -75.690308.
The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.
The geograpical coordinate of Winnipeg, Canada are 49.884017, -97.168579.
The geograpical coordinate of Saskatoon, Canada are 52.131802, -106.660767.
The geograpical coordinate of Whitehorse, Canada are 60.721571, -135.054932.
The geograpical coordinate of Yellowknife, Canada are 62.4709394, -114.4384901.
The geograpical coordinate of Iqaluit, Canada are 63.7577822, -68.5587568.
The geograpical coordinate of St. John's, Canada are 47.561701, -52.715149.
The geo

In [150]:
print(lat)
print(lng)

[49.2608724, 45.4972159, 40.7308619, 34.0536834, -23.5506507, -22.9110137, 19.4326009, -12.0621065, -33.4377968, -34.6075616]
[-123.1139529, -73.6103642, -73.9871558, -118.2427669, -46.6333824, -43.2093727, -99.1333416, -77.0365256, -70.6504451, -58.437076]


In [4]:
city = ['Vancouver, Canada', 'Montreal, Canada', 'Quebec City, Canada', 'Edmonton, Canada', 'Calgary, Canada',
             'Ottawa, Canada', 'Toronto, Canada', 'Winnipeg, Canada', 'Saskatoon, Canada', 'Whitehorse, Canada',
             'Yellowknife, Canada', 'Iqaluit, Canada', 'St. John\'s, Canada', 'Fredericton, Canada']
country = ['Canada', 'Canada','Canada', 'Canada','Canada', 'Canada','Canada', 'Canada','Canada', 'Canada',
           'Canada', 'Canada','Canada', 'Canada',]

In [5]:
d = {'City': city, 'Country': country, 'Latitude': lat, 'Longitude': lng}
df = pd.DataFrame(data=d)
df

,City,Country,Latitude,Longitude
0,"Vancouver, Canada",Canada,49.260872,-123.113953
1,"Montreal, Canada",Canada,45.497216,-73.610364
2,"Quebec City, Canada",Canada,46.825960,-71.235223
3,"Edmonton, Canada",Canada,53.535411,-113.507996
4,"Calgary, Canada",Canada,51.025327,-114.049868
5,"Ottawa, Canada",Canada,45.421106,-75.690308
6,"Toronto, Canada",Canada,43.653963,-79.387207
7,"Winnipeg, Canada",Canada,49.884017,-97.168579
8,"Saskatoon, Canada",Canada,52.131802,-106.660767
9,"Whitehorse, Canada",Canada,60.721571,-135.054932


In [6]:
CLIENT_ID = 'YW1G4YM4YVJ0BV3J0D2P1BXGESTULQYIWET3D1I5Z31Q4EA1'
CLIENT_SECRET = 'OI033BPVLR2ZEGO4MANA3DYGOM451QDTKNKZFIPLJZV1VZR5'
VERSION = '20180605'

### Now we set up the request from the Free Foursquare API (with a limit of 100 venues per city) to collect the venues data.

In [7]:
LIMIT = 100

radius = 10000 
section = 'food'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius,
    section,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YW1G4YM4YVJ0BV3J0D2P1BXGESTULQYIWET3D1I5Z31Q4EA1&client_secret=OI033BPVLR2ZEGO4MANA3DYGOM451QDTKNKZFIPLJZV1VZR5&v=20180605&ll=45.966425,-66.645813&radius=10000&section=food&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c268fe74434b91ea4f0e6d3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Fredericton',
  'headerFullLocation': 'Fredericton',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 46.05642509000009,
    'lng': -66.51657308813023},
   'sw': {'lat': 45.87642490999991, 'lng': -66.77505291186978}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c2e0e6ae760c9b69bdf4549',
       'name': 'Palate Restaurant & Cafe',
       'location': {'address': '462 Queen St',
        'lat': 45.96233821565255,
        'lng': -66.64177617467865,
        'labeledLatLngs': [{'label': 'display',
          

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Palate Restaurant & Cafe,Restaurant,45.962338,-66.641776
1,Chess Piece Pâtisserie & Cafe,Café,45.963354,-66.644017
2,New England Pizza,Pizza Place,45.967675,-66.629905
3,Cafe Loka & Bistro,Café,45.957570,-66.647978
4,Luna Pizza,Italian Restaurant,45.962246,-66.643788


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

69 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section=food&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                             'City Latitude', 
                             'City Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [ ]:
cities_restaurants = getNearbyVenues(names = df['City'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude'])

Vancouver, Canada
Montreal, Canada
Quebec City, Canada


In [201]:
print(cities_restaurants.shape)
cities_restaurants.head()

(982, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Vancouver,49.260872,-123.113953,La Taqueria Pinche Taco Shop,49.263380,-123.112707,Taco Place
1,Vancouver,49.260872,-123.113953,Peaceful Restaurant 和平饭店,49.263086,-123.115808,Chinese Restaurant
2,Vancouver,49.260872,-123.113953,Indian Roti Kitchen,49.258950,-123.115087,Indian Restaurant
3,Vancouver,49.260872,-123.113953,Vij's,49.257525,-123.115083,Indian Restaurant
4,Vancouver,49.260872,-123.113953,Marulilu Cafe,49.263338,-123.114118,Japanese Restaurant


### Then we check the number of different food venues categories and check how many times they appear.

In [172]:
print('There are {} uniques categories.'.format(len(cities_restaurants['Venue Category'].unique())))

There are 94 uniques categories.


In [173]:
# one hot encoding
restaurants_onehot = pd.get_dummies(cities_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
restaurants_onehot['City'] = cities_restaurants['City'] 

# move neighborhood column to the first column
fixed_columns = [restaurants_onehot.columns[-1]] + list(restaurants_onehot.columns[:-1])
restaurants_onehot = restaurants_onehot[fixed_columns]

restaurants_onehot.head()

,City,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoshoku Restaurant
0,Vancouver,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Vancouver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Vancouver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Vancouver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Vancouver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
restaurants_onehot.shape

(982, 95)

In [175]:
restaurants_mean = restaurants_onehot.groupby('City').mean().reset_index()
restaurants_mean

,City,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoshoku Restaurant
0,Buenos Aires,0.01,0.00000,0.00,0.11,0.010000,0.07000,0.01,0.090000,0.00,...,0.00,0.01,0.000000,0.02,0.00,0.00,0.00,0.03,0.00,0.00
1,Lima,0.00,0.02439,0.00,0.00,0.012195,0.02439,0.00,0.060976,0.00,...,0.00,0.00,0.012195,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Los Angeles,0.00,0.02000,0.00,0.00,0.010000,0.02000,0.00,0.040000,0.00,...,0.02,0.00,0.000000,0.01,0.00,0.00,0.00,0.03,0.00,0.01
3,Mexico City,0.00,0.00000,0.00,0.01,0.000000,0.01000,0.00,0.070000,0.00,...,0.13,0.02,0.000000,0.00,0.00,0.00,0.00,0.01,0.00,0.00
4,Montreal,0.00,0.00000,0.00,0.00,0.030000,0.01000,0.01,0.060000,0.00,...,0.00,0.00,0.000000,0.01,0.00,0.01,0.00,0.01,0.08,0.00
5,New York City,0.00,0.11000,0.00,0.01,0.020000,0.01000,0.02,0.020000,0.00,...,0.01,0.03,0.000000,0.02,0.00,0.00,0.01,0.04,0.02,0.00
6,Rio de Janeiro,0.00,0.01000,0.00,0.00,0.010000,0.01000,0.00,0.030000,0.00,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Santiago,0.00,0.00000,0.01,0.00,0.020000,0.00000,0.00,0.020000,0.00,...,0.00,0.00,0.000000,0.01,0.01,0.00,0.00,0.05,0.00,0.00
8,São Paulo,0.00,0.00000,0.00,0.00,0.020000,0.00000,0.00,0.060000,0.00,...,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.08,0.00,0.00
9,Vancouver,0.00,0.00000,0.00,0.00,0.000000,0.03000,0.02,0.080000,0.02,...,0.03,0.00,0.000000,0.01,0.00,0.00,0.00,0.04,0.02,0.00


In [176]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now we can start to build our dataframe with the most frequent food venues in each city.

In [178]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Food Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Food Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = restaurants_mean['City']

for ind in np.arange(restaurants_mean.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(restaurants_mean.iloc[ind, :], num_top_venues)
   

cities_venues_sorted

,City,1st Most Common Food Venue,2nd Most Common Food Venue,3rd Most Common Food Venue
0,Buenos Aires,Argentinian Restaurant,Pizza Place,Café
1,Lima,Peruvian Restaurant,Chinese Restaurant,Seafood Restaurant
2,Los Angeles,Japanese Restaurant,Sushi Restaurant,Italian Restaurant
3,Mexico City,Mexican Restaurant,Taco Place,Café
4,Montreal,Chinese Restaurant,Vietnamese Restaurant,Café
5,New York City,Italian Restaurant,American Restaurant,Pizza Place
6,Rio de Janeiro,Brazilian Restaurant,Italian Restaurant,Pizza Place
7,Santiago,Sandwich Place,Café,Pizza Place
8,São Paulo,Japanese Restaurant,Brazilian Restaurant,Café
9,Vancouver,Japanese Restaurant,Bakery,Sushi Restaurant


### Now we seek comparisons with the demographic data found on the internet.
For example, at Canadas government website https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/imm/Table.cfm?Lang=E&T=21&Geo=01&SO=4D, we can find information about the foreign population in Canada, and we can see that from 27,245 japanese immigrants 13,360 live in British Columbia (Vancouver province), that means that from all the japanese immigrants living in the entire country, allmost half live in the Vancouver Province, which can explain the most common food venue being Japanese.

Notebook by Francisco Tosetto da Silva, made for the Capstone Project of the IBM Data Science Professional Certificate.